<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/extract/gradioprompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install easyocr gradio pythainlp langchain langchain_huggingface langchain_community pytesseract
!sudo apt-get install ghostscript

In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os

resume_directory = '/content/drive/MyDrive/AIEngineer/resume/resume_LLM'
pdf_files = [os.path.join(resume_directory, f) for f in os.listdir(resume_directory) if f.endswith('.pdf')]

# Print the list of PDF files to verify
print(pdf_files)

['/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai1.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai7.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/Resume ba2.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/Resume ba4.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai6.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/Resume ba3.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/Resume ba1.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/Resume ba5.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume ai8.pdf', '/content/drive/MyDrive/AIEngineer/resume/resume-test/resume hr1.pdf', '/con

## **Job Qualification**

| **ᴊᴏʙ** | **ᴘᴏꜱɪᴛɪᴏɴ** | **ᴄᴏᴜɴᴛ** |
|:---:|:---:|:---:|
| 1 | ʙᴜꜱɪɴᴇꜱꜱ ᴀɴᴀʟʏꜱᴛ | 5 |
| 2 | ᴀɪ ᴇɴɢɪɴᴇᴇʀ | 7 |


In [74]:
import os
import re
import numpy as np
import pandas as pd
import easyocr
import subprocess
from pythainlp.phayathaibert.core import NamedEntityTagger
import gradio as gr
from PIL import Image
import requests
from threading import Semaphore
from concurrent.futures import ThreadPoolExecutor, as_completed
from huggingface_hub import InferenceClient
import pytesseract

In [101]:
resume_prompt = """
YOU ARE AN EXPERT CAREER COACH AND RESUME EVALUATOR, TASKED WITH REVIEWING A CANDIDATE'S RESUME BASED ON A SPECIFIC JOB DESCRIPTION. YOUR ROLE IS TO ASSESS WHETHER THE RESUME MEETS THE REQUIREMENTS OF THE JOB, CLASSIFYING IT INTO ONE OF THREE CATEGORIES: "PASS," "BORDERLINE," OR "NOT PASS."

### INSTRUCTIONS ###
- CAREFULLY READ AND UNDERSTAND THE JOB DESCRIPTION TO IDENTIFY ALL CRITICAL REQUIREMENTS (SKILLS, EXPERIENCE, QUALIFICATIONS, ETC.).
- ANALYZE THE RESUME TO EVALUATE HOW WELL IT MATCHES THE REQUIRED SKILLS AND EXPERIENCE.
- DETERMINE WHETHER THE CANDIDATE FULLY MEETS, PARTIALLY MEETS, OR DOES NOT MEET THE JOB REQUIREMENTS.
- CLASSIFY THE RESUME AS "PASS," "BORDERLINE," OR "NOT PASS" BASED ON THE FOLLOWING CRITERIA:

#### **Classification Criteria**:
1. **PASS**: The resume clearly demonstrates that the candidate meets or exceeds all key job requirements. The candidate's experience and skills are highly aligned with the job description.
2. **BORDERLINE**: The resume shows that the candidate meets some of the requirements, but there are gaps in key areas, such as relevant experience, skills, or qualifications.
3. **NOT PASS**: The resume does not sufficiently meet the job requirements. The candidate lacks the necessary experience, skills, or qualifications for the position.

#### **CHAIN OF THOUGHT** (Reasoning Process):
1. **Step 1**: Review the **Job Description** and extract the key required skills, experience, and qualifications.
2. **Step 2**: Compare the candidate's **Resume** to the job requirements:
   - Identify relevant skills, experiences, or qualifications mentioned in the resume.
   - Highlight any missing or insufficiently demonstrated requirements.
3. **Step 3**: Classify the resume as "Pass," "Borderline," or "Not Pass" based on how well it matches the job requirements.
4. **Step 4**: Provide a concise rationale for your decision, explaining which requirements were met or unmet.

### **Job Description:**
{jobdescribe}

### **Resume:**
{resume}

### **Evaluation**:
- **Classification**: (Pass / Borderline / Not Pass)
- **Rationale**: Provide a brief explanation detailing how well the resume aligns with the job description. Mention any strengths or gaps in experience, skills, and qualifications.

### **Few-Shot Example (never copy it)**:
### **Job Description:**
- Required: 5+ years in software development, experience with Python and JavaScript, familiarity with cloud platforms, strong communication skills, and a bachelor’s degree in Computer Science or a related field.

### **Resume:**
- Candidate has 6 years of experience in software development, is proficient in Python but lacks strong JavaScript experience. They have worked with cloud platforms (AWS) and hold a degree in Computer Science. Communication skills are well-demonstrated through experience leading team projects.

### **Evaluation**:
- **Classification**: Borderline
- **Rationale**: The candidate meets most of the key requirements (software development experience, Python proficiency, cloud platform familiarity, and education). However, they lack sufficient JavaScript experience, which is a critical requirement for this role.
"""

In [102]:
# Function to convert PDF to images using Ghostscript
def convert_pdf_to_images(pdf_file, output_folder="/content/images", dpi=300):
    try:
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        output_format = os.path.join(output_folder, "page_%03d.png")
        gs_command = [
            "gs",
            "-sDEVICE=png16m",
            f"-r{dpi}",
            "-o", output_format,
            pdf_file
        ]
        subprocess.run(gs_command, check=True)
        image_files = sorted([os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith(".png")])
        return image_files
    except subprocess.CalledProcessError as e:
        print(f"Ghostscript error: {e}")
        return []

# Function to extract text from a single image
def extract_text_from_image(image_path):
    reader = easyocr.Reader(['th', 'en'], gpu=True)
    image = Image.open(image_path)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    sorted_data = sorted(result, key=lambda x: x[0][0][1])
    plain_text = "\n".join([text for _, text, _ in sorted_data])
    return plain_text

# Function to split text into chunks based on token length
def chunk_text(text, tokenizer, max_tokens=510):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk = tokenizer.convert_tokens_to_string(tokens[i:i+max_tokens])
        chunks.append(chunk)
    return chunks

# Tag and clean text with chunking
def tag_and_clean_text(text, tagger, tokenizer, unwanted_pattern):
    text_chunks = chunk_text(text, tokenizer)

    tagged_text = ""
    cleaned_text = ""

    for chunk in text_chunks:
        try:
            ner = tagger.get_ner(chunk, tag=True)
            pattern = r'<(?!ORGANIZATION|PERCENT|TIME)[^>]+>[^<]+</[^>]+>'
            cleaned_ner = re.sub(pattern, '', ner)
            cleaned_ner = re.sub(r'</?(ORGANIZATION|PERCENT|TIME)>', '', cleaned_ner)
            cleaned_ner = re.sub(unwanted_pattern, '', cleaned_ner, flags=re.IGNORECASE)
            cleaned_ner = re.sub(r'\bal\b', 'ai', cleaned_ner, flags=re.IGNORECASE)

            tagged_text += ner + "\n"
            cleaned_text += cleaned_ner + "\n"
        except Exception as e:
            print(f"Error processing chunk: {e}")

    return tagged_text.strip(), cleaned_text.strip()

# Function to generate an answer using the prompt
def generate_answer(resume, job):
    formatted_prompt = resume_prompt.replace("{resume}", resume).replace("{jobdescribe}", job)
    client = InferenceClient('https://ai-api.manageai.co.th/llm-model-02/')
    response = client.text_generation(formatted_prompt, **model_params)
    output = "".join(response)
    return output

def extract_classification(text):
    pattern = r'\*\*\s*Classification\s*\**:\s*(?:\[)?\s*(Pass|Borderline|Not Pass)\s*(?:\])?'
    match = re.search(pattern, text, re.IGNORECASE)
    if not match:
        pattern = r'Classification\s*:\s*(?:\[)?\s*(Pass|Borderline|Not Pass)\s*(?:\])?'
        match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    return text

def process_file(file, job, is_pdf=True):
    if is_pdf:
        images = convert_pdf_to_images(file.name)
        raw_text = ""
        for image_path in images:
            raw_text += extract_text_from_image(image_path) + "\n"
    else:
        raw_text = extract_text_from_image(file.name)

    tagger = NamedEntityTagger()
    tokenizer = tagger.tokenizer

    unwanted_terms = [
        'ที่อยู่', 'โทรศัพท์', 'อีเมล', 'linkedin', ':', ',', '-', '|',
        'ประวัติส่วนตัว', 'เกี่ยวกับฉัน', 'about me', 'ชื่อ', 'สกุล', 'tell', 'โทร', 'โทรงาน',
        'ชื่อเล่น', 'อายุ', 'วันเกิด', 'พุทธ', 'ศาสนา', 'สัญชาติ', 'phone',
        'ช่องทางการติดต่อ', '_', 're sume', 'resume', 'resu me'
    ]
    unwanted_pattern = '|'.join(map(re.escape, unwanted_terms))

    tagged_text, cleaned_text = tag_and_clean_text(raw_text, tagger, tokenizer, unwanted_pattern)
    evaluation = generate_answer(cleaned_text, job)
    result = extract_classification(evaluation)

    df = pd.DataFrame([{
        "File": os.path.basename(file.name),
        "Raw_Text": raw_text,
        "Tagged_Text": tagged_text,
        "Cleaned_Text": cleaned_text,
        "Evaluation": evaluation,
        "Result": result
    }])
    return df

def process_multiple_files(files, job, is_pdf=True):
    all_results = pd.DataFrame()
    for file in files:
        df = process_file(file, job, is_pdf)
        all_results = pd.concat([all_results, df], ignore_index=True)
    return all_results

# Function to save DataFrame to CSV
def save_dataframe(df):
    try:
        csv_path = '/content/output.csv'
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        return csv_path
    except Exception as e:
        print(f"Error saving DataFrame: {e}")
        return ""

# Semaphore to limit concurrent API requests
semaphore = Semaphore(50)

# Define the model parameters
model_params = {
    "max_new_tokens": 510,
    "temperature": 0.1,
    "top_p": 0.95,
    "repetition_penalty": 1.0
}

# Function to process a batch of DataFrame rows
def generate_prompt_answer_optimized(row):
    resume = row.get('Cleaned_Text', "")
    job = row.get('Job_Description', "")
    result = generate_answer(resume, job)
    return result

# Function to process a batch of rows
def process_batch(batch_df):
    return [generate_prompt_answer_optimized(row) for _, row in batch_df.iterrows()]

# Batch processing with threading
def batch_process(df, batch_size=32):
    results = [None] * len(df)
    with ThreadPoolExecutor() as executor:
        futures = {}
        for i in range(0, len(df), batch_size):
            batch_df = df.iloc[i:i + batch_size]
            future = executor.submit(process_batch, batch_df)
            futures[future] = (i, i + batch_size)

        for future in as_completed(futures):
            start_idx, end_idx = futures[future]
            batch_results = future.result()
            results[start_idx:end_idx] = batch_results

    return results

def gradio_interface(files, job, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
    try:
        model_params.update({
            "max_new_tokens": max_new_tokens,
            "temperature": temperature,
            "top_p": top_p,
            "repetition_penalty": repetition_penalty
        })

        # Determine if files include PDFs and/or JPGs
        is_pdf_file = is_pdf
        df = process_multiple_files(files, job, is_pdf_file)
        df['Evaluation'] = batch_process(df, batch_size=32)
        df['Result'] = df['Evaluation'].apply(extract_classification)

        # Select only the required columns for output
        df_output = df[['Raw_Text', 'Evaluation', 'Result']]

        csv_path = save_dataframe(df)
        return df_output, csv_path
    except Exception as e:
        print(f"Error in gradio_interface: {e}")
        return pd.DataFrame(), ""


# Define the Gradio interface
def create_gradio_interface():
    with gr.Blocks() as demo:
        with gr.Row():
            with gr.Column(scale=1):
                file_input = gr.Files(label="Upload Files", file_count="multiple")
                job_input = gr.Textbox(placeholder="Enter the job description here...", label="Job Description", lines=10)
                is_pdf_input = gr.Checkbox(label="PDF", value=True)

                # Model parameters
                max_new_tokens_input = gr.Slider(label="Max New Tokens", minimum=50, maximum=1024, value=512, step=50)
                temperature_input = gr.Slider(label="Temperature", minimum=0.01, maximum=1.0, value=0.7, step=0.1)
                top_p_input = gr.Slider(label="Top P", minimum=0.1, maximum=1.0, value=0.95, step=0.05)
                repetition_penalty_input = gr.Slider(label="Repetition Penalty", minimum=0.1, maximum=2.0, value=1.0, step=0.1)

                submit_btn = gr.Button("Process")

            with gr.Column(scale=2):
                # Modify to display only Raw_Text, Evaluation, and Result columns
                output_df = gr.DataFrame(headers=["Raw_Text", "Evaluation", "Result"], type="pandas", interactive=True)
                output_csv = gr.File(label="Download CSV")

            # Function to process inputs and generate output
            def on_submit(files, job, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty):
                df, csv_path = gradio_interface(files, job, is_pdf, max_new_tokens, temperature, top_p, repetition_penalty)
                return df, csv_path

            submit_btn.click(on_submit,
                             inputs=[file_input, job_input, is_pdf_input, max_new_tokens_input, temperature_input, top_p_input, repetition_penalty_input],
                             outputs=[output_df, output_csv])

    return demo


In [ ]:
app = create_gradio_interface()
app.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://27e99a55672cbf1b38.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/usr/local/lib/python3.10/dist-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(trained_model, ma

In [104]:
import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI

,File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Job Description:\n- Required: 3+ years of expe...,Pass
1,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ ประวัติการศึกษา มหาวิทยาลั...,Job Description:\n- Required: 3+ years of expe...,Pass
2,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Job Description:\n- Required: 4+ years of expe...,Pass


In [106]:
print(AI['Raw_Text'].iloc[2])

re sume
ประวัติส่วนตัว
090-123-4567
 pimchanok@example. com
linkedin .com/in/pimchanoksuksawat
 เกี่ยวกับฉัน
ผู้ |เชี่ยวชาญด้าน al ที่มีประสบการณ์ 4 เขี
พิมพ์ชนก สุขสวัสดิ์
ในด้านการพัฒนาโซลูชันการเรียนรู้ของ
 วิศวกรปัญญาประดิษฐ์
 สำหรับการวิเคราะห์ ข้อมูล
เครืองและ al
โดยใช้ scikit learn และ keras มี
ธุรกิจ
ทักษะ
ความสามารถ
i.
ความสามารถในการจัดการข้อมูลขนาด
ใหญ่และสร้างโมเดลที่ช่วยเพิ่ม
ภาษา1 โปรแกรม: python, c++
ประสิทธิภาพในการตัดสินใจเชิงธุรกิจ
การประมวลผลภาพ: opencv, pillow
การเรียนรู้เชิงลึก: tensorflow, keras
 ประวัติการทำงาน
การจัดการข้อมูล: sol, solite
 วิศวกร al กุมภาพันธ์ 2020
ปัจจุบัน
-
พัฒนาโมเดล machine learning เพือ
 ประกาศนียบัตร
การพยากรณ์ทางการเงินที่ช่วยให้การ
ตัดสินใจเชิงกลยุทธ์มีความแม่นยำขึ้น
 google cloud professional data
engineer
30%
 coursera machine learning by
 ร่วมมือกับทีม|วิเคราะห์ ข้อมูลเพือทำการ
จัดการข้อมูลและสร้างรายงานที่มี
 stanford
 ประสิทธิภาพ สำหรับผู้บริหาร
ข้อมูลการศึกษา
ผู้ ช่วยวิจัย al
มัธยมศึกษา
ศึกษานารี
มิถุนายน 2019
มิถุนายน 2017
_
 อุดม

In [105]:
print(AI['Evaluation'].iloc[2])

Job Description:
- Required: 4+ years of experience in AI development, proficiency in Python and C++, ability to handle large data sets, experience with machine learning frameworks (Scikit-learn, TensorFlow, Keras), knowledge of image processing (OpenCV, Pillow), and a bachelor’s degree in Computer Science or a related field.

Resume:
- The candidate has 4 years of experience in AI development, specifically in developing AI solutions for data analysis. They are proficient in Python and C++, and have experience with Scikit-learn and Keras. They have skills in handling large data sets and creating models that improve business decision-making. They have experience with image processing using OpenCV and Pillow. They have a background in machine learning and deep learning using TensorFlow and Keras. They have a current role as a data engineer, developing machine learning models for financial forecasting to improve strategic decision-making. They have collaborated with data analysis teams to

In [23]:
import pandas as pd
AI = pd.read_csv('/content/output.csv')
AI

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,RESUME AI ENG.pdf,"narakorn vorakul\n18/4 pak nam, mmueang samut ...",<PERSON>nar</PERSON><PERSON>akorn vorakul</PER...,address email.v ai engineer ตําแหน่ง birth da...,Strengths: [List the candidate's strengths as ...,Borderline
1,RESUME AI TH.pdf,นางสาวนารากร วรกุล\nที่อยู่:\n18/4 ถ.ศรีสมุทร ...,<PERSON>น</PERSON><PERSON>นางสาว</PERSON><PERS...,ai engineer ตําแหน่ง แนะนําตัวเบื้องต้น ผู้ ส...,Skills: [List the key skills that the candidat...,Not Pass
2,resume ai1.pdf,สมหญิง\n แก้วกาจญ์\n วิศวกรปัญญาประดิษฐ์\nal ท...,<PERSON>ส</PERSON><PERSON>สม</PERSON><PERSON>ห...,วิศวกรปัญญาประดิษฐ์ ai ที่มีประสบการณ์ 3 ปีในก...,Additional Notes: [Provide any additional note...,Pass
3,resume ai2.pdf,วงศ์เจริญ\nณ็ชพล\n วิศวกรปัญญาประดิษฐ์\n ประว...,<PERSON>วงศ์เจริญ ณ็ชพล</PERSON> วิศวกรปัญญาปร...,วิศวกรปัญญาประดิษฐ์ ประวัติการศึกษา มหาวิทยาลั...,Qualifications: [Score / 10]\nSkills: [Score /...,Pass
4,resume ai3.pdf,re sume\nประวัติส่วนตัว\n090-123-4567\n pimcha...,re sume ประวัติส่วนตัว<PHONE> 09</PHONE><PHONE...,ผู้ เชี่ยวชาญด้าน ai ที่มีประสบการณ์ 4 เขี ในด...,Strengths: [List at least two strong qualifica...,Pass
5,resume ai4.pdf,อาทิตย์\nรัตนวิจิตร\n วิศวกรปัญญาประดิษฐ์\nปร...,<PERSON>อ</PERSON><PERSON>อาทิตย์ รัตนวิจิตร</...,วิศวกรปัญญาประดิษฐ์ ประสบการณํ ทํางาน วิศวกร a...,Skills & Experience: [Score out of 10]\nCultur...,Pass
6,resume ai5.pdf,ศิรินทิพย์\nศรีวัฒนกิจ\n วิศวกรปัญญาประดิษฐ์\n...,<PERSON>ศ</PERSON><PERSON>ศิริ</PERSON><PERSON...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์ทํางาน วิศวกร a...,Standout Qualifications: [List any qualificati...,Pass
7,resume ai6.pdf,linkedin:\nนภัสสร วิวัฒนาวงศ์\nlinkedin.com i...,linkedin:<PERSON> </PERSON><PERSON>น</PERSON><...,in วิศวกรปัญญาประดิษฐ์ ประสบการณ์การทํางาน วิ...,Skills Alignment: [Assess the alignment of the...,Pass
8,resume ai7.pdf,ปัญญา วิริยะชัย\n วิศวกรปัญญาประดิษฐ์\nประสบกา...,<PERSON>ป</PERSON><PERSON>ปัญญา วิริยะชัย</PER...,วิศวกรปัญญาประดิษฐ์ ประสบการณ์การฝึกงาน บัณฑิต...,Strengths: [List the candidate's strengths tha...,Pass
9,resume ai8.pdf,alexander jones\nartificial intelligence engin...,alexander jones artificial intelligence engine...,alexander jones artificial inigence engineer i...,Qualifications: [Evaluate the match between th...,Pass


In [32]:
print(AI['Tagged_Text'].iloc[0])

<PERSON>nar</PERSON><PERSON>akorn vorakul</PERSON><LOCATION> 18</LOCATION><LOCATION>/4 pak nam</LOCATION><LOCATION>,</LOCATION><LOCATION> mmueang</LOCATION><LOCATION> samut prakan</LOCATION><LOCATION> district</LOCATION><LOCATION>,</LOCATION><LOCATION> samut prakan</LOCATION><ZIP> 10</ZIP><ZIP>270</ZIP> address:<PHONE> 09</PHONE><PHONE>9</PHONE><PHONE>-827-8887</PHONE> phone: email:<PERSON> nar</PERSON><PERSON>akorn</PERSON>.v<EMAIL>@</EMAIL><EMAIL>kkumail.com</EMAIL> al engineer ตําแหน่ง: birth date:<DATE> april 23, 2002</DATE> summary enthusiastic and detail oriented recent graduate with a bachelor's degree in statistics and data science, looking for an appropriate position. demonstrates strong analytical skills, attention to detail, and proficiency in python. experienced in data cleaning, preprocessing, machine learning implementation, and web application development with python., gained through academic projects and internships. education may bachelor of science (statistics and dat

In [ ]:
import pandas as pd
BA = pd.read_csv('/content/output.csv')
BA

,PDF File,Raw_Text,Tagged_Text,Cleaned_Text,Evaluation,Result
0,Resume BA 2.pdf,"พิมพ์นารา วัฒนชัย\nที่อยู่: 789 ถนนรัชดาภิเษก,...",<PERSON>พ</PERSON><PERSON>พิมพ์</PERSON><PERSO...,st เป้าหมายในการทํางาน นําความเชี่ยวชาญด้านการ...,Skills Assessment: \n- [Skill 1]: [Level of p...,Borderline
1,Resume ba 5.pdf,081-456-7890\nนัทธมน ฺรัตนโชติ\n nattamon. rad...,<PHONE>08</PHONE><PERSON>1-4</PERSON><PERSON>5...,เป้าหมายในการ มุ่งมันในการนําทักษะด้านการจัดกา...,Skills & Qualifications: [Assess the skills an...,Pass
2,Resume BA 3.pdf,จิตรลดา\n086-234-5678\njitlada.s@domain. com\...,<PERSON>จ</PERSON><PERSON>จิตร</PERSON><PERSON...,เสริม business analyst เป้าหมายในการทํางาน ต้อ...,Job Description Requirements: [List the key re...,Pass
3,Resume ba2.pdf,ศุภกิจฺ มงคลชัย\n089-234-5678\n supakit .m@do...,<PERSON>ศ</PERSON><PERSON>ศุภ</PERSON><PERSON>...,ประสบการณ์การ เป้าหมายในการทํางาน flnancial an...,### **Job Description:**\nWe are seeking an ex...,Pass
4,Resume baa.pdf,"ธนกร อินทรีย์พงษ์\n ที่อยู่:\n456 ถนนพระราม 3,...",<PERSON>ธ</PERSON><PERSON>ธน</PERSON><PERSON>ก...,เป้าหมายในการทํางาน มุ่งมันที่จะใช้ความรู้และท...,Specific Skill Set: [List the specific skills ...,Pass
